<h2 style="color:blue" align="center">Plain Deep Neural Network (DNN)</h2>

#### Import the necessary libraries

In [88]:
import tensorflow as tf 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import time, math 
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras 
from keras.models import Sequential
from keras.layers import Dense 
from keras.callbacks import Callback, CSVLogger 

#### Load and split the dataset into training and testing set

In [89]:
(X_train,y_train),(X_test,y_test) = keras.datasets.boston_housing.load_data(path="boston_housing.npz", test_split=0.2, seed=3)

#### Normalize the input dataset

In [90]:
mms1 = MinMaxScaler()
mms1.fit(X_train)
X_train_normalized = mms1.transform(X_train)
mms2 = MinMaxScaler()
mms2.fit(X_test)
X_test_normalized = mms2.transform(X_test) 

#### Define DNN model with two layers, optimizer, metrics, and loss function

In [91]:
np.random.seed(3)

def get_model(): 
    model = Sequential([
        # input layer 784 neurons to first hidden layer with 64 neurons
        Dense(100, input_dim = 13, activation='relu'), 
        # first hidden layer to second hidden layer
        Dense(100, activation='relu'),  
        # Output layer with 10 neurons
        Dense(1, activation='linear')
    ])
    
    # General SGD
    #opti = keras.optimizers.SGD(learning_rate=0.01)
    
    # SGD with momentum
    #opti = keras.optimizers.SGD(learning_rate=0.01, momentum=0.6)
    
    # SGD with Nesterov momentum 
    #opti = keras.optimizers.SGD(learning_rate=0.01, momentum=0.6, nesterov=True)
    
    # RMSprop 
    #opti = keras.optimizers.RMSprop(learning_rate=0.001, momentum=0.6)
    
    # Adam
    opti = keras.optimizers.Adam(learning_rate=0.001) 
    
    # Adamax
    #opti = keras.optimizers.Adamax(learning_rate=0.001) 
    
    model.compile(
        optimizer = opti,
        loss = 'mean_squared_error',
        metrics = ['mae']
    )
    
    return model

#### Custom callbacks

For HM based optimizer model

In [92]:
class CustomCallbackHM(Callback):  
    
    initial_weights = 0
    previous_weights = 0
    call_hm = 0 
     
    def on_train_begin(self, logs=None):
        self.initial_weights = model_hm.get_weights() 
        self.initial_weights = np.array(self.initial_weights,dtype=object)
        self.previous_weights = self.initial_weights
        # Harmonic mean based weights calculation
        self.call_hm = np.vectorize(self.apply_hm)  
             
    def on_epoch_end(self, epoch, logs={}): 
        counter = 0
        num_layers = len(model_hm.layers)  
        current_weights = model_hm.get_weights()
        current_weights = np.array(current_weights,dtype=object)        

        for i in range(num_layers):  
            # Harmonic mean based weights calculation
            current_weights[counter] = self.call_hm(self.previous_weights[counter], current_weights[counter])
            counter = counter + 2
            
        # Updating the model with new weights
        updated = current_weights.tolist()   
        model_hm.set_weights(updated)
        self.previous_weights = current_weights
        
    def apply_hm(self, v1, v2):     
        if v1==0 or v2==0:
            return v2
        elif v1>0 and v2>0:
            hm = 2*v1*v2/(v1+v2)
            min1 = min(v1,v2)
            diff = abs(hm-min1)
            if v2 > v1:
                return v2 + diff
            else:
                return v2 - diff
        elif v1<0 and v2<0:
            hm = 2*v1*v2/(v1+v2)
            max1 = max(v1,v2)
            diff = abs(hm-max1)
            if v2 > v1:
                return v2 + diff
            else:
                return v2 - diff
        else:
            return v2   

To record loss and accuracy in CSV file

In [93]:
logger_generic_model = CSVLogger('Generic_model_MNIST.csv', append=False, separator=',')
logger_hm_model = CSVLogger('HM_model_MNIST.csv', append=False, separator=',')

#### Training

Generic opimizer model

In [94]:
model_wihtout_hm = get_model() 
st = time.time()
model_wihtout_hm.fit(X_train_normalized, y_train, epochs = 500, verbose=1, callbacks=[logger_generic_model], batch_size=404) 
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

Epoch 1/500
1/1 [==============================] - 0s 311ms/step - loss: 610.7894 - mae: 22.8991
Epoch 2/500
1/1 [==============================] - 0s 8ms/step - loss: 607.1183 - mae: 22.8143
Epoch 3/500
1/1 [==============================] - 0s 11ms/step - loss: 603.4476 - mae: 22.7290
Epoch 4/500
1/1 [==============================] - 0s 11ms/step - loss: 599.7672 - mae: 22.6430
Epoch 5/500
1/1 [==============================] - 0s 11ms/step - loss: 596.0605 - mae: 22.5561
Epoch 6/500
1/1 [==============================] - 0s 11ms/step - loss: 592.3169 - mae: 22.4680
Epoch 7/500
1/1 [==============================] - 0s 8ms/step - loss: 588.5312 - mae: 22.3786
Epoch 8/500
1/1 [==============================] - 0s 11ms/step - loss: 584.7053 - mae: 22.2879
Epoch 9/500
1/1 [==============================] - 0s 9ms/step - loss: 580.8412 - mae: 22.1961
Epoch 10/500
1/1 [==============================] - 0s 11ms/step - loss: 576.9252 - mae: 22.1029
Epoch 11/500
1/1 [=======================

1/1 [==============================] - 0s 9ms/step - loss: 50.8441 - mae: 5.0091
Epoch 173/500
1/1 [==============================] - 0s 10ms/step - loss: 50.6231 - mae: 4.9969
Epoch 174/500
1/1 [==============================] - 0s 10ms/step - loss: 50.4034 - mae: 4.9846
Epoch 175/500
1/1 [==============================] - 0s 11ms/step - loss: 50.1851 - mae: 4.9724
Epoch 176/500
1/1 [==============================] - 0s 9ms/step - loss: 49.9680 - mae: 4.9603
Epoch 177/500
1/1 [==============================] - 0s 7ms/step - loss: 49.7518 - mae: 4.9483
Epoch 178/500
1/1 [==============================] - 0s 10ms/step - loss: 49.5366 - mae: 4.9363
Epoch 179/500
1/1 [==============================] - 0s 11ms/step - loss: 49.3223 - mae: 4.9244
Epoch 180/500
1/1 [==============================] - 0s 7ms/step - loss: 49.1088 - mae: 4.9127
Epoch 181/500
1/1 [==============================] - 0s 8ms/step - loss: 48.8960 - mae: 4.9012
Epoch 182/500
1/1 [==============================] - 0s 10m

1/1 [==============================] - 0s 7ms/step - loss: 33.1210 - mae: 3.9769
Epoch 259/500
1/1 [==============================] - 0s 11ms/step - loss: 32.9312 - mae: 3.9640
Epoch 260/500
1/1 [==============================] - 0s 8ms/step - loss: 32.7422 - mae: 3.9516
Epoch 261/500
1/1 [==============================] - 0s 7ms/step - loss: 32.5541 - mae: 3.9392
Epoch 262/500
1/1 [==============================] - 0s 11ms/step - loss: 32.3670 - mae: 3.9269
Epoch 263/500
1/1 [==============================] - 0s 10ms/step - loss: 32.1807 - mae: 3.9147
Epoch 264/500
1/1 [==============================] - 0s 6ms/step - loss: 31.9954 - mae: 3.9025
Epoch 265/500
1/1 [==============================] - 0s 9ms/step - loss: 31.8111 - mae: 3.8903
Epoch 266/500
1/1 [==============================] - 0s 9ms/step - loss: 31.6277 - mae: 3.8780
Epoch 267/500
1/1 [==============================] - 0s 7ms/step - loss: 31.4454 - mae: 3.8656
Epoch 268/500
1/1 [==============================] - 0s 10ms/

Epoch 344/500
1/1 [==============================] - 0s 9ms/step - loss: 21.7909 - mae: 3.1429
Epoch 345/500
1/1 [==============================] - 0s 10ms/step - loss: 21.7231 - mae: 3.1378
Epoch 346/500
1/1 [==============================] - 0s 9ms/step - loss: 21.6565 - mae: 3.1330
Epoch 347/500
1/1 [==============================] - 0s 13ms/step - loss: 21.5912 - mae: 3.1283
Epoch 348/500
1/1 [==============================] - 0s 12ms/step - loss: 21.5273 - mae: 3.1238
Epoch 349/500
1/1 [==============================] - 0s 11ms/step - loss: 21.4645 - mae: 3.1198
Epoch 350/500
1/1 [==============================] - 0s 9ms/step - loss: 21.4029 - mae: 3.1160
Epoch 351/500
1/1 [==============================] - 0s 8ms/step - loss: 21.3423 - mae: 3.1123
Epoch 352/500
1/1 [==============================] - 0s 10ms/step - loss: 21.2826 - mae: 3.1086
Epoch 353/500
1/1 [==============================] - 0s 11ms/step - loss: 21.2240 - mae: 3.1046
Epoch 354/500
1/1 [=========================

Epoch 430/500
1/1 [==============================] - 0s 10ms/step - loss: 18.5521 - mae: 2.9280
Epoch 431/500
1/1 [==============================] - 0s 8ms/step - loss: 18.5289 - mae: 2.9264
Epoch 432/500
1/1 [==============================] - 0s 9ms/step - loss: 18.5058 - mae: 2.9247
Epoch 433/500
1/1 [==============================] - 0s 6ms/step - loss: 18.4827 - mae: 2.9228
Epoch 434/500
1/1 [==============================] - 0s 7ms/step - loss: 18.4596 - mae: 2.9210
Epoch 435/500
1/1 [==============================] - 0s 6ms/step - loss: 18.4365 - mae: 2.9191
Epoch 436/500
1/1 [==============================] - 0s 10ms/step - loss: 18.4136 - mae: 2.9173
Epoch 437/500
1/1 [==============================] - 0s 10ms/step - loss: 18.3906 - mae: 2.9155
Epoch 438/500
1/1 [==============================] - 0s 7ms/step - loss: 18.3674 - mae: 2.9137
Epoch 439/500
1/1 [==============================] - 0s 10ms/step - loss: 18.3442 - mae: 2.9118
Epoch 440/500
1/1 [===========================

HM based optimizer model

In [95]:
model_hm = get_model() 
st = time.time()
model_hm.fit(X_train_normalized, y_train, epochs = 380, verbose=1, callbacks=[CustomCallbackHM(),logger_hm_model], batch_size=404) 
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

Epoch 1/380
1/1 [==============================] - 0s 342ms/step - loss: 602.4933 - mae: 22.7436
Epoch 2/380
1/1 [==============================] - 0s 19ms/step - loss: 596.5667 - mae: 22.6095
Epoch 3/380
1/1 [==============================] - 0s 28ms/step - loss: 590.4710 - mae: 22.4706
Epoch 4/380
1/1 [==============================] - 0s 25ms/step - loss: 584.2515 - mae: 22.3280
Epoch 5/380
1/1 [==============================] - 0s 26ms/step - loss: 577.8912 - mae: 22.1818
Epoch 6/380
1/1 [==============================] - 0s 19ms/step - loss: 571.4087 - mae: 22.0318
Epoch 7/380
1/1 [==============================] - 0s 23ms/step - loss: 564.7717 - mae: 21.8770
Epoch 8/380
1/1 [==============================] - 0s 25ms/step - loss: 557.9363 - mae: 21.7162
Epoch 9/380
1/1 [==============================] - 0s 25ms/step - loss: 550.8658 - mae: 21.5483
Epoch 10/380
1/1 [==============================] - 0s 29ms/step - loss: 543.5349 - mae: 21.3727
Epoch 11/380
1/1 [====================

1/1 [==============================] - 0s 19ms/step - loss: 39.6058 - mae: 4.4165
Epoch 172/380
1/1 [==============================] - 0s 27ms/step - loss: 39.3682 - mae: 4.4037
Epoch 173/380
1/1 [==============================] - 0s 25ms/step - loss: 39.1311 - mae: 4.3908
Epoch 174/380
1/1 [==============================] - 0s 23ms/step - loss: 38.8946 - mae: 4.3778
Epoch 175/380
1/1 [==============================] - 0s 21ms/step - loss: 38.6585 - mae: 4.3647
Epoch 176/380
1/1 [==============================] - 0s 22ms/step - loss: 38.4228 - mae: 4.3514
Epoch 177/380
1/1 [==============================] - 0s 22ms/step - loss: 38.1876 - mae: 4.3380
Epoch 178/380
1/1 [==============================] - 0s 22ms/step - loss: 37.9537 - mae: 4.3245
Epoch 179/380
1/1 [==============================] - 0s 28ms/step - loss: 37.7206 - mae: 4.3108
Epoch 180/380
1/1 [==============================] - 0s 23ms/step - loss: 37.4870 - mae: 4.2967
Epoch 181/380
1/1 [==============================] - 0

1/1 [==============================] - 0s 36ms/step - loss: 18.1514 - mae: 2.8645
Epoch 342/380
1/1 [==============================] - 0s 23ms/step - loss: 18.1191 - mae: 2.8614
Epoch 343/380
1/1 [==============================] - 0s 21ms/step - loss: 18.0865 - mae: 2.8583
Epoch 344/380
1/1 [==============================] - 0s 27ms/step - loss: 18.0526 - mae: 2.8553
Epoch 345/380
1/1 [==============================] - 0s 21ms/step - loss: 18.0174 - mae: 2.8514
Epoch 346/380
1/1 [==============================] - 0s 22ms/step - loss: 17.9802 - mae: 2.8468
Epoch 347/380
1/1 [==============================] - 0s 21ms/step - loss: 17.9306 - mae: 2.8398
Epoch 348/380
1/1 [==============================] - 0s 25ms/step - loss: 17.8642 - mae: 2.8302
Epoch 349/380
1/1 [==============================] - 0s 24ms/step - loss: 17.7860 - mae: 2.8195
Epoch 350/380
1/1 [==============================] - 0s 19ms/step - loss: 17.7248 - mae: 2.8121
Epoch 351/380
1/1 [==============================] - 0

Model summary

In [96]:
model_hm.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_60 (Dense)            (None, 100)               1400      
                                                                 
 dense_61 (Dense)            (None, 100)               10100     
                                                                 
 dense_62 (Dense)            (None, 1)                 101       
                                                                 
Total params: 11,601
Trainable params: 11,601
Non-trainable params: 0
_________________________________________________________________


###### Testing the model

Generic opimizer model

In [97]:
model_wihtout_hm.evaluate(X_test_normalized, y_test)

4/4 [==============================] - 0s 2ms/step - loss: 20.3269 - mae: 3.0890


[20.326906204223633, 3.0889501571655273]

In [98]:
input_data = np.array([0.63796, 0.00, 8.140, 0, 0.5380, 6.0960, 84.50, 4.4619, 4, 307.0, 21.00, 380.02, 10.26])
input_data_transformed = mms1.transform(input_data.reshape(1, -1))
output = model_wihtout_hm.predict(input_data_transformed) 
output[0][0]

1/1 [==============================] - 0s 48ms/step


19.629356

HM based optimizer model

In [99]:
model_hm.evaluate(X_test_normalized, y_test)

4/4 [==============================] - 0s 2ms/step - loss: 20.9398 - mae: 3.0403


[20.939834594726562, 3.0402965545654297]

In [101]:
output = model_hm.predict(input_data_transformed) 
output[0][0]

1/1 [==============================] - 0s 20ms/step


20.117882